In [42]:
%run 01_Model_CFE.ipynb


ADS: frdp_c --> AMOR + ppi_c
CPR1_CYP71AV1: AMOR + h_c + nadph_c + o2_c --> AAOH + h2o_c + nadp_c
ADH1: AAOH + nad_c --> AAld + h_c + nadh_c
ALDH1: AAld + h2o_c + nadp_c --> AA + h_c + nadph_c


In [43]:
iMM904s=iMM904.copy()
#changing source to sucrose
medium09 = iMM904s.medium
medium09['EX_sucr_e'] = 10
medium09['EX_glc__D_e'] = 0
iMM904s.medium = medium09

In [44]:

from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield

from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization, NADH_NADPH

iMM904s.solver = "glpk" # what is this?
biomass = iMM904s.reactions.BIOMASS_SC5_notrace
biomass.lower_bound = 0.1 # OBS denne skal måske ikke være 0,1 tjek tutorial notebooken og forstå det
demand = iMM904s.reactions.DM_AA

iMM904s.objective = demand
product_yield = product_yield(demand, iMM904s.reactions.EX_sucr_e)

In [45]:
CofactorSwapOptimization?

Init signature:
CofactorSwapOptimization(
    cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']),
    candidate_reactions=None,
    skip_reactions=None,
    *args,
    **kwargs,
)
Docstring:     
Optimize co-factor swapping

As suggested in [1]_, flux through a given reaction can sometimes be optimized by swapping complementary
co-factor. This class implements a search for reactions when swapped improve the given objective. Briefly,
the approach is to

- find reactions that have all the targeted co-factor pairs e.g. (nad_c -> nadp_c, nadh_c -> nadph_c)

- add reactions that have the co-factors swapped and then by a search algorithm switching one off in favor of the
  other

The implementation here differs from that in [1]_ in that we use a general purpose search algorithm rather than
formulating the search as a mixed integer linear programming problem.

References
----------
.. [1] King, Zachary A., and Adam M. Feist. "Optimizing Cofactor Specificity of Oxidoreductase Enzym

In [46]:
swap_opt = CofactorSwapOptimization(model=iMM904s, objective_function=product_yield, plot=True, cofactor_id_swaps=(['nadp_c', 'nadph_c'],['nad_c', 'nadh_c'] ))


In [47]:
%%time
result = swap_opt.run(max_size=5)

Starting optimization at Fri, 01 Dec 2023 13:24:48


HBox()

Finished after 00:00:21
Wall time: 49.4 s


In [48]:
result

,index,targets,fitness
0,0,"(ALCD2x_copy2,)",0.180861
1,2,"(ALCD2x_copy1,)",0.180861
2,5,"(GAPD,)",0.180861
3,6,"(HMGCOAR,)",0.177793
4,7,"(ADH1, CPR1_CYP71AV1)",0.154841
5,8,"(FAS180, ADH1)",0.150019
6,9,"(CPR1_CYP71AV1,)",0.150002
7,10,"(ADH1,)",0.149914
8,11,"(C3STKR2, IPMD, GLUDy)",0.147435
9,12,"(GLUDy, HISTD, DHFR)",0.147373


In [49]:
iMM904.reactions.GAPD

Reaction identifier,GAPD
Name,Glyceraldehyde-3-phosphate dehydrogenase
Memory address,0x01c365323b80
Stoichiometry,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c Glyceraldehyde 3-phosphate + Nicotinamide adenine dinucleotide + Phosphate <=> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,YJL052W or YJR009C or YGR192C
Lower bound,-999999.0
Upper bound,999999.0
